In [1]:
import pycisTopic
import glob
import os

# os.environ["TMPDIR"] = "dirty_tmp"
import pybiomart as pbm
import pandas as pd
import pickle
from pycisTopic.qc import *
from IPython.display import Image, display
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

import multiprocess as mp  # for kde multithreading calculation
from multiprocess import Pool
from ray.dashboard import *

%matplotlib inline
%load_ext lab_black

In [2]:
import collections as cl
import gc
import logging
import sys
from typing import Dict, List, Optional, Tuple, Union

import matplotlib.backends.backend_pdf
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pyranges as pr
import ray
import seaborn as sns
from scipy.stats import gaussian_kde, norm

from pycisTopic.cistopic_class import *
from pycisTopic.utils import (
    collapse_duplicates,
    multiplot_from_generator,
    read_fragments_from_file,
)

In [3]:
import importlib

# Download annotation

In [4]:
os.chdir(
    "/dodrio/scratch/projects/starting_2022_023/benchmark/software/software/anaconda3/lib/python3.9/site-packages"
)
import ray

print(ray.__version__)
wdir = "/dodrio/scratch/projects/starting_2022_023/benchmark/scatac_benchmark/fixedcells_4_merged"
os.chdir(wdir)

2.0.0


In [5]:
import sys

print(ray.__version__)

2.0.0


In [6]:
!pwd

/dodrio/scratch/projects/starting_2022_023/benchmark/scatac_benchmark/fixedcells_4_merged


In [7]:
wdir = "/dodrio/scratch/projects/starting_2022_023/benchmark/scatac_benchmark/fixedcells_4_merged"
os.chdir(wdir)

In [8]:
genome = "hg38"

pbm_genome_name_dict = {
    "hg38": "hsapiens_gene_ensembl",
    "hg37": "hsapiens_gene_ensembl",
    "mm10": "mmusculus_gene_ensembl",
    "dm6": "dmelanogaster_gene_ensembl",
}

pbm_host_dict = {
    "hg38": "http://www.ensembl.org",
    "hg37": "http://grch37.ensembl.org/",
    "mm10": "http://nov2020.archive.ensembl.org/",
    "dm6": "http://www.ensembl.org",
}

if os.path.exists(f"annotation.tsv"):
    print(f"Loading cached genome annotation...")
    annotation = pd.read_csv("annotation.tsv", sep="\t", header=0, index_col=0)
else:
    dataset = pbm.Dataset(name=pbm_genome_name_dict[genome], host=pbm_host_dict[genome])

    annotation = dataset.query(
        attributes=[
            "chromosome_name",
            "transcription_start_site",
            "strand",
            "external_gene_name",
            "transcript_biotype",
        ]
    )
    filter = annotation["Chromosome/scaffold name"].str.contains("CHR|GL|JH|MT")
    annotation = annotation[~filter]
    annotation["Chromosome/scaffold name"] = annotation[
        "Chromosome/scaffold name"
    ].str.replace(r"(\b\S)", r"chr\1")
    annotation.columns = ["Chromosome", "Start", "Strand", "Gene", "Transcript_type"]
    annotation = annotation[annotation.Transcript_type == "protein_coding"]
    annotation.to_csv("annotation.tsv", sep="\t")

Loading cached genome annotation...


In [9]:
annotation

,Chromosome,Start,Strand,Gene,Transcript_type
37,chrKI270442chr.chr1,380608,1,5S_rRNA,rRNA
38,chrKI270711chr.chr1,24650,-1,NaN,protein_coding
39,chrKI270711chr.chr1,29626,-1,NaN,protein_coding
84,chrKI270442chr.chr1,217401,-1,5_8S_rRNA,rRNA
516,chrKI270744chr.chr1,51114,-1,U6,snRNA
...,...,...,...,...,...
45278,chr11,33076149,1,LINC00294,lncRNA
45279,chr11,55262155,1,TRIM48,protein_coding
45280,chr12,262836,-1,SLC6A13,protein_coding
45281,chr12,224431,-1,SLC6A13,processed_transcript


In [10]:
fragments_list = sorted(glob.glob("../1_data_repository/fixedcells_fragments/*.tsv.gz"))
fragments_dict = {}
for fragments_file in fragments_list:
    sample = fragments_file.split("/")[-1].split(".fragments.tsv.gz")[0]
    fragments_dict[sample] = fragments_file
fragments_dict

{'BIO_ddseq_1.FIXEDCELLS': '../1_data_repository/fixedcells_fragments/BIO_ddseq_1.FIXEDCELLS.fragments.tsv.gz',
 'BIO_ddseq_2.FIXEDCELLS': '../1_data_repository/fixedcells_fragments/BIO_ddseq_2.FIXEDCELLS.fragments.tsv.gz',
 'BIO_ddseq_3.FIXEDCELLS': '../1_data_repository/fixedcells_fragments/BIO_ddseq_3.FIXEDCELLS.fragments.tsv.gz',
 'BIO_ddseq_4.FIXEDCELLS': '../1_data_repository/fixedcells_fragments/BIO_ddseq_4.FIXEDCELLS.fragments.tsv.gz',
 'BRO_mtscatac_1.FIXEDCELLS': '../1_data_repository/fixedcells_fragments/BRO_mtscatac_1.FIXEDCELLS.fragments.tsv.gz',
 'BRO_mtscatac_2.FIXEDCELLS': '../1_data_repository/fixedcells_fragments/BRO_mtscatac_2.FIXEDCELLS.fragments.tsv.gz',
 'CNA_10xmultiome_1.FIXEDCELLS': '../1_data_repository/fixedcells_fragments/CNA_10xmultiome_1.FIXEDCELLS.fragments.tsv.gz',
 'CNA_10xmultiome_2.FIXEDCELLS': '../1_data_repository/fixedcells_fragments/CNA_10xmultiome_2.FIXEDCELLS.fragments.tsv.gz',
 'CNA_10xv11_1.FIXEDCELLS': '../1_data_repository/fixedcells_fragmen

In [11]:
fragments_dict.pop("VIB_hydrop_11.FIXEDCELLS")
fragments_dict.pop("VIB_hydrop_12.FIXEDCELLS")
fragments_dict.pop("VIB_hydrop_21.FIXEDCELLS")
fragments_dict.pop("VIB_hydrop_22.FIXEDCELLS")

'../1_data_repository/fixedcells_fragments/VIB_hydrop_22.FIXEDCELLS.fragments.tsv.gz'

In [12]:
master_set_path = "../fixedcells_3_cistopic_consensus/master_peaks/all.FIXEDCELLS.master_peaks.occurrence_filtered9.bed"

In [13]:
regions_paths_dict = {x: master_set_path for x in fragments_dict.keys()}
regions_paths_dict

{'BIO_ddseq_1.FIXEDCELLS': '../fixedcells_3_cistopic_consensus/master_peaks/all.FIXEDCELLS.master_peaks.occurrence_filtered9.bed',
 'BIO_ddseq_2.FIXEDCELLS': '../fixedcells_3_cistopic_consensus/master_peaks/all.FIXEDCELLS.master_peaks.occurrence_filtered9.bed',
 'BIO_ddseq_3.FIXEDCELLS': '../fixedcells_3_cistopic_consensus/master_peaks/all.FIXEDCELLS.master_peaks.occurrence_filtered9.bed',
 'BIO_ddseq_4.FIXEDCELLS': '../fixedcells_3_cistopic_consensus/master_peaks/all.FIXEDCELLS.master_peaks.occurrence_filtered9.bed',
 'BRO_mtscatac_1.FIXEDCELLS': '../fixedcells_3_cistopic_consensus/master_peaks/all.FIXEDCELLS.master_peaks.occurrence_filtered9.bed',
 'BRO_mtscatac_2.FIXEDCELLS': '../fixedcells_3_cistopic_consensus/master_peaks/all.FIXEDCELLS.master_peaks.occurrence_filtered9.bed',
 'CNA_10xmultiome_1.FIXEDCELLS': '../fixedcells_3_cistopic_consensus/master_peaks/all.FIXEDCELLS.master_peaks.occurrence_filtered9.bed',
 'CNA_10xmultiome_2.FIXEDCELLS': '../fixedcells_3_cistopic_consensus/ma

In [14]:
len(regions_paths_dict)

47

Now, make a sub dictionary of all samples within the fragments dict that have not been run yet (good for resuming a stopped cistopic run):

In [15]:
cistopic_qc_out = os.path.join(wdir, "cistopic_qc_out_MASTER")
if not os.path.exists(cistopic_qc_out):
    os.makedirs(cistopic_qc_out)

In [16]:
fragments_sub_dict = {}
regions_sub_dict = {}
for sample in regions_paths_dict.keys():
    metadata_file = os.path.join(cistopic_qc_out, sample + "__metadata_bc.pkl")
    print(f"Checking if {metadata_file} exist...")
    if os.path.exists(metadata_file):
        print("\tMetadata exists! Skipping...")
    else:
        fragments_sub_dict[sample] = fragments_dict[sample]
        print("\tMetadata does not exist, adding to subdict to generate")

Checking if /dodrio/scratch/projects/starting_2022_023/benchmark/scatac_benchmark/fixedcells_4_merged/cistopic_qc_out_MASTER/BIO_ddseq_1.FIXEDCELLS__metadata_bc.pkl exist...
	Metadata does not exist, adding to subdict to generate
Checking if /dodrio/scratch/projects/starting_2022_023/benchmark/scatac_benchmark/fixedcells_4_merged/cistopic_qc_out_MASTER/BIO_ddseq_2.FIXEDCELLS__metadata_bc.pkl exist...
	Metadata does not exist, adding to subdict to generate
Checking if /dodrio/scratch/projects/starting_2022_023/benchmark/scatac_benchmark/fixedcells_4_merged/cistopic_qc_out_MASTER/BIO_ddseq_3.FIXEDCELLS__metadata_bc.pkl exist...
	Metadata does not exist, adding to subdict to generate
Checking if /dodrio/scratch/projects/starting_2022_023/benchmark/scatac_benchmark/fixedcells_4_merged/cistopic_qc_out_MASTER/BIO_ddseq_4.FIXEDCELLS__metadata_bc.pkl exist...
	Metadata does not exist, adding to subdict to generate
Checking if /dodrio/scratch/projects/starting_2022_023/benchmark/scatac_benchmar

In [17]:
regions_sub_dict = {x: regions_paths_dict[x] for x in sorted(fragments_sub_dict.keys())}
regions_sub_dict

{'BIO_ddseq_1.FIXEDCELLS': '../fixedcells_3_cistopic_consensus/master_peaks/all.FIXEDCELLS.master_peaks.occurrence_filtered9.bed',
 'BIO_ddseq_2.FIXEDCELLS': '../fixedcells_3_cistopic_consensus/master_peaks/all.FIXEDCELLS.master_peaks.occurrence_filtered9.bed',
 'BIO_ddseq_3.FIXEDCELLS': '../fixedcells_3_cistopic_consensus/master_peaks/all.FIXEDCELLS.master_peaks.occurrence_filtered9.bed',
 'BIO_ddseq_4.FIXEDCELLS': '../fixedcells_3_cistopic_consensus/master_peaks/all.FIXEDCELLS.master_peaks.occurrence_filtered9.bed',
 'BRO_mtscatac_1.FIXEDCELLS': '../fixedcells_3_cistopic_consensus/master_peaks/all.FIXEDCELLS.master_peaks.occurrence_filtered9.bed',
 'BRO_mtscatac_2.FIXEDCELLS': '../fixedcells_3_cistopic_consensus/master_peaks/all.FIXEDCELLS.master_peaks.occurrence_filtered9.bed',
 'CNA_10xmultiome_1.FIXEDCELLS': '../fixedcells_3_cistopic_consensus/master_peaks/all.FIXEDCELLS.master_peaks.occurrence_filtered9.bed',
 'CNA_10xmultiome_2.FIXEDCELLS': '../fixedcells_3_cistopic_consensus/ma

In [18]:
def compute_qc_stats_single(
    fragments,
    tss_annotation: Union[pd.DataFrame, pr.PyRanges],
    stats: Optional[List[str]] = [
        "barcode_rank_plot",
        "duplicate_rate",
        "insert_size_distribution",
        "profile_tss",
        "frip",
    ],
    label: Optional[str] = None,
    path_to_regions: Optional[str] = None,
    valid_bc: Optional[List[str]] = None,
    n_frag: Optional[int] = None,
    n_bc: Optional[int] = None,
    tss_flank_window: Optional[int] = 1000,
    tss_window: Optional[int] = 50,
    tss_minimum_signal_window: Optional[int] = 100,
    tss_rolling_window: Optional[int] = 10,
    min_norm: Optional[int] = 0.2,
    partition: Optional[int] = 1,
    check_for_duplicates: Optional[bool] = True,
    remove_duplicates: Optional[bool] = True,
    use_polars: Optional[bool] = True
):
    """
    Wrapper function to compute QC statistics on several samples. For detailed instructions, please see the independent functions.
    Parameters
    ---
    fragments: str
            Path to fragments file.
    tss_annotation: pd.DataFrame or pr.PyRanges
            A data frame or pyRanges containing transcription start sites for each gene, with 'Chromosome', 'Start' and 'Strand' as columns (additional columns will be ignored).
    stats: list, optional
            A list with the statistics that have to be computed. Default: All ('barcode_rank_plot', 'duplicate_rate', 'insert_size_distribution', 'profile_tss', 'FRIP).
    label: str
            Sample label. Default: None.
    path_to_regions: str
            Path to regions file to use for FRIP.
    valid_bc: list, optional
            A list containing selected barcodes. This parameter is ignored if n_frag or n_bc are specified. Default: None.
    n_frag: int, optional
            Minimal number of fragments assigned to a barcode to be kept. Either n_frag or n_bc can be specified. Default: None.
    n_bc: int, optional
            Number of barcodes to select. Either n_frag or n_bc can be specified. Default: None.
    tss_window: int, optional
            Window around the TSS used to count fragments in the TSS when calculating the TSS enrichment per barcode. Default: 50 (+/- 50 bp).
    tss_flank_window: int, optional
            Flanking window around the TSS. Default: 1000 (+/- 1000 bp).
    tss_minimum_signal_window: int, optional
            Tail window use to normalize the TSS enrichment. Default: 100 (average signal in the 100bp in the extremes of the TSS window).
    tss_rolling_window: int, optional
            Rolling window used to smooth signal. Default: 10.
    min_norm: int, optional
            Minimum normalization score. If the average minimum signal value is below this value, this number is used to normalize the TSS signal. This approach penalizes cells with fewer reads.
    check_for_duplicates: bool, optional
            If no duplicate counts are provided per row in the fragments file, whether to collapse duplicates. Default: True.
    remove_duplicates: bool, optional
            Whether to remove duplicates. Default: True.
    use_polars: bool, optional
            Whether to use polars to read fragments files. Default: True.
    Return
    ---
    pd.DataFrame or list and list
            A list with the barcode statistics for all samples (or a combined data frame with a column 'Sample' indicating the sample of origin) and a list of dictionaries with the sample-level profiles for each sample.
    """

    # Create logger
    level = logging.INFO
    log_format = "%(asctime)s %(name)-12s %(levelname)-8s %(message)s"
    handlers = [logging.StreamHandler(stream=sys.stdout)]
    logging.basicConfig(level=level, format=log_format, handlers=handlers)
    log = logging.getLogger("cisTopic")
    # Compute stats
    metrics = {}
    metadata_bc_dict = {}
    profile_data_dict = {}
    # Prepare fragments
    if isinstance(fragments, str):
        log.info("Reading " + label)
        fragments_df = read_fragments_from_file(fragments, use_polars=use_polars).df
    else:
        fragments_df = fragments
    # Convert to category for memory efficiency
    fragments_df["Name"] = fragments_df["Name"].astype("category")
    # Check for duplicates
    if "Score" not in fragments_df or all(fragments_df["Score"] == "."):
        fragments_df = fragments_df[["Chromosome", "Start", "End", "Name"]]
        if check_for_duplicates:
            log.info("Collapsing duplicates")
            fragments_df = pd.concat(
                [
                    collapse_duplicates(fragments_df[fragments_df.Chromosome == x])
                    for x in fragments_df.Chromosome.cat.categories.values
                ]
            )
        else:
            fragments_df["Score"] = 1
    else:
        fragments_df = fragments_df[["Chromosome", "Start", "End", "Name", "Score"]]
    fragments_df["Score"] = fragments_df["Score"].astype("int32")
    # Prepare valid barcodes
    if valid_bc is not None:
        if n_bc is not None or n_frag is not None:
            valid_bc = None
    # Rank plot
    if "barcode_rank_plot" in stats:
        # Rank plot
        log.info("Computing barcode rank plot for " + label)
        metrics["barcode_rank_plot"] = barcode_rank_plot(
            fragments=fragments_df,
            valid_bc=valid_bc,
            n_frag=n_frag,
            n_bc=n_bc,
            remove_duplicates=remove_duplicates,
            plot=False,
            return_bc=True,
            return_plot_data=True,
        )
        if valid_bc is None:
            fragments_df = fragments_df[
                fragments_df.Name.isin(set(metrics["barcode_rank_plot"]["valid_bc"]))
            ]

    gc.collect()
    # Duplicate rate
    if "duplicate_rate" in stats:
        # Duplicate rate
        log.info("Computing duplicate rate plot for " + label)
        metrics["duplicate_rate"] = duplicate_rate(
            fragments=fragments_df, valid_bc=valid_bc, plot=False, return_plot_data=True
        )

    gc.collect()
    # Fragment size
    if "insert_size_distribution" in stats:
        # Fragment size
        log.info("Computing insert size distribution for " + label)
        metrics["insert_size_distribution"] = insert_size_distribution(
            fragments=fragments_df,
            valid_bc=valid_bc,
            remove_duplicates=remove_duplicates,
            plot=False,
            return_plot_data=True,
        )
    fragments_df = pr.PyRanges(fragments_df)
    gc.collect()
    # TSS
    if "profile_tss" in stats:
        # TSS
        log.info("Computing TSS profile for " + label)
        profile_tss_metrics = profile_tss(
            fragments=fragments_df,
            annotation=tss_annotation,
            valid_bc=valid_bc,
            plot=False,
            n_cpu=1,
            partition=partition,
            flank_window=tss_flank_window,
            tss_window=tss_window,
            minimum_signal_window=tss_minimum_signal_window,
            rolling_window=tss_rolling_window,
            min_norm=min_norm,
            return_TSS_enrichment_per_barcode=True,
            return_TSS_coverage_matrix_per_barcode=True,
            return_plot_data=True,
        )
        if profile_tss_metrics is not None:
            metrics["profile_tss"] = profile_tss_metrics
    gc.collect()
    # FRIP
    if "frip" in stats:
        # FRIP
        log.info("Computing FRIP profile for " + label)
        metrics["frip"] = frip(
            fragments=fragments_df,
            path_to_regions=path_to_regions,
            valid_bc=valid_bc,
            remove_duplicates=remove_duplicates,
            n_cpu=1,
            plot=False,
            return_plot_data=True,
        )
    del fragments_df
    gc.collect()
    metadata_bc, profile_data = metrics2data(metrics)

    if isinstance(metadata_bc, pd.DataFrame):
        metadata_bc = metadata_bc.fillna(0)

    metadata_bc_dict = {label: metadata_bc}
    profile_data_dict = {label: profile_data}
    log.info("Sample " + label + " done!")

    return metadata_bc_dict, profile_data_dict

In [19]:
for sample in regions_sub_dict.keys():
    metadata_bc, profile_data = compute_qc_stats_single(
        fragments=fragments_sub_dict[sample],
        tss_annotation=annotation,
        stats=[
            "barcode_rank_plot",
            "duplicate_rate",
            "insert_size_distribution",
            "profile_tss",
            "frip",
        ],
        label = sample,
        path_to_regions=regions_sub_dict[sample],
        valid_bc=None,
        n_frag=10,
        n_bc=None,
        tss_flank_window=2000,
        tss_window=50,
        tss_minimum_signal_window=100,
        tss_rolling_window=10,
        # min_norm=0.2,
        remove_duplicates=True,
    )
    with open(
        os.path.join(cistopic_qc_out, f"{sample}__metadata_bc.pkl"), "wb"
    ) as f:
        pickle.dump(metadata_bc[sample], f, protocol=4)

    with open(
        os.path.join(cistopic_qc_out, f"{sample}__profile_data.pkl"), "wb"
    ) as f:
        pickle.dump(profile_data[sample], f, protocol=4)
else:
    print("All samples already processed.")

2022-09-20 23:17:22,013 cisTopic     INFO     Reading BIO_ddseq_1.FIXEDCELLS
2022-09-21 00:21:24,047 cisTopic     INFO     Computing barcode rank plot for BIO_ddseq_1.FIXEDCELLS
2022-09-21 00:21:24,112 cisTopic     INFO     Counting fragments
2022-09-21 00:29:22,772 cisTopic     INFO     Marking barcodes with more than 10
2022-09-21 00:29:38,550 cisTopic     INFO     Returning plot data
2022-09-21 00:29:39,124 cisTopic     INFO     Returning valid barcodes
2022-09-21 00:32:07,001 cisTopic     INFO     Computing duplicate rate plot for BIO_ddseq_1.FIXEDCELLS
2022-09-21 00:46:10,549 cisTopic     INFO     Return plot data
2022-09-21 00:46:17,703 cisTopic     INFO     Computing insert size distribution for BIO_ddseq_1.FIXEDCELLS
2022-09-21 00:46:17,726 cisTopic     INFO     Counting fragments
2022-09-21 00:46:56,627 cisTopic     INFO     Returning plot data
2022-09-21 01:04:09,801 cisTopic     INFO     Computing TSS profile for BIO_ddseq_1.FIXEDCELLS
2022-09-21 01:10:40,118 cisTopic     IN

In [20]:
ray.shutdown()

In [21]:
n_cores = 1
if regions_sub_dict != {}:
    samples_sub = list(regions_sub_dict.keys())
    blocks = [samples_sub[i : i + n_cores] for i in range(0, len(samples_sub), n_cores)]
    for samples_torun_in_block in blocks:
        fragments_sub_dict_block = {
            key: fragments_sub_dict[key] for key in samples_torun_in_block
        }
        regions_sub_dict_block = {
            key: regions_sub_dict[key] for key in samples_torun_in_block
        }

        metadata_bc_dict, profile_data_dict = compute_qc_stats(
            fragments_dict=fragments_sub_dict_block,
            tss_annotation=annotation,
            stats=[
                "barcode_rank_plot",
                "duplicate_rate",
                "insert_size_distribution",
                "profile_tss",
                "frip",
            ],
            label_list=None,
            path_to_regions=regions_sub_dict_block,
            n_cpu=n_cores,
            valid_bc=None,
            n_frag=10,
            n_bc=None,
            tss_flank_window=2000,
            tss_window=50,
            tss_minimum_signal_window=100,
            tss_rolling_window=10,
            # min_norm=0.2,
            remove_duplicates=True,
        )

        ray.shutdown()
        print(f"Dumping files in {cistopic_qc_out}...")
        for sample in sorted(metadata_bc_dict.keys()):
            metadata_bc_dict[sample]["sample_id"] = sample
            metadata_bc_dict[sample].index = [
                x + "___" + sample for x in list(metadata_bc_dict[sample].index)
            ]
            with open(
                os.path.join(cistopic_qc_out, f"{sample}__metadata_bc.pkl"), "wb"
            ) as f:
                pickle.dump(metadata_bc_dict[sample], f, protocol=4)

            with open(
                os.path.join(cistopic_qc_out, f"{sample}__profile_data.pkl"), "wb"
            ) as f:
                pickle.dump(profile_data_dict[sample], f, protocol=4)
else:
    print("All samples already processed.")

2022-09-21 10:36:18,707	INFO worker.py:1509 -- Started a local Ray instance. View the dashboard at http://127.0.0.1:8265 


(compute_qc_stats_ray pid=2741380) 2022-09-21 10:36:28,652 cisTopic     INFO     Reading BIO_ddseq_1.FIXEDCELLS
(compute_qc_stats_ray pid=2741380) 2022-09-21 10:38:46,910 cisTopic     INFO     Computing barcode rank plot for BIO_ddseq_1.FIXEDCELLS
(compute_qc_stats_ray pid=2741380) 2022-09-21 10:38:46,910 cisTopic     INFO     Counting fragments
(compute_qc_stats_ray pid=2741380) 2022-09-21 10:38:59,615 cisTopic     INFO     Marking barcodes with more than 10
(compute_qc_stats_ray pid=2741380) 2022-09-21 10:39:00,019 cisTopic     INFO     Returning plot data
(compute_qc_stats_ray pid=2741380) 2022-09-21 10:39:00,033 cisTopic     INFO     Returning valid barcodes
(compute_qc_stats_ray pid=2741380) 2022-09-21 10:39:02,639 cisTopic     INFO     Computing duplicate rate plot for BIO_ddseq_1.FIXEDCELLS
(compute_qc_stats_ray pid=2741380) 2022-09-21 10:39:19,601 cisTopic     INFO     Return plot data
(compute_qc_stats_ray pid=2741380) 2022-09-21 10:39:19,707 cisTopic     INFO     Computing in

2022-09-21 10:43:32,077	INFO worker.py:1509 -- Started a local Ray instance. View the dashboard at http://127.0.0.1:8265 


(compute_qc_stats_ray pid=2741960) 2022-09-21 10:43:35,804 cisTopic     INFO     Reading BIO_ddseq_2.FIXEDCELLS
(compute_qc_stats_ray pid=2741960) 2022-09-21 10:45:13,366 cisTopic     INFO     Computing barcode rank plot for BIO_ddseq_2.FIXEDCELLS
(compute_qc_stats_ray pid=2741960) 2022-09-21 10:45:13,367 cisTopic     INFO     Counting fragments
(compute_qc_stats_ray pid=2741960) 2022-09-21 10:45:23,882 cisTopic     INFO     Marking barcodes with more than 10
(compute_qc_stats_ray pid=2741960) 2022-09-21 10:45:24,344 cisTopic     INFO     Returning plot data
(compute_qc_stats_ray pid=2741960) 2022-09-21 10:45:24,362 cisTopic     INFO     Returning valid barcodes
(compute_qc_stats_ray pid=2741960) 2022-09-21 10:45:26,298 cisTopic     INFO     Computing duplicate rate plot for BIO_ddseq_2.FIXEDCELLS
(compute_qc_stats_ray pid=2741960) 2022-09-21 10:45:40,957 cisTopic     INFO     Return plot data
(compute_qc_stats_ray pid=2741960) 2022-09-21 10:45:41,060 cisTopic     INFO     Computing in

2022-09-21 10:49:23,886	INFO worker.py:1509 -- Started a local Ray instance. View the dashboard at http://127.0.0.1:8265 


...

(compute_qc_stats_ray pid=2762583) 2022-09-21 13:49:22,172 cisTopic     INFO     Reading VIB_10xv2_2.FIXEDCELLS
(compute_qc_stats_ray pid=2762583) 2022-09-21 13:50:18,590 cisTopic     INFO     Computing barcode rank plot for VIB_10xv2_2.FIXEDCELLS
(compute_qc_stats_ray pid=2762583) 2022-09-21 13:50:18,590 cisTopic     INFO     Counting fragments
(compute_qc_stats_ray pid=2762583) 2022-09-21 13:50:20,799 cisTopic     INFO     Marking barcodes with more than 10
(compute_qc_stats_ray pid=2762583) 2022-09-21 13:50:20,837 cisTopic     INFO     Returning plot data
(compute_qc_stats_ray pid=2762583) 2022-09-21 13:50:20,840 cisTopic     INFO     Returning valid barcodes
(compute_qc_stats_ray pid=2762583) 2022-09-21 13:50:23,249 cisTopic     INFO     Computing duplicate rate plot for VIB_10xv2_2.FIXEDCELLS
(compute_qc_stats_ray pid=2762583) 2022-09-21 13:50:26,595 cisTopic     INFO     Return plot data
(compute_qc_stats_ray pid=2762583) 2022-09-21 13:50:26,674 cisTopic     INFO     Computing in

2022-09-21 13:52:16,457	INFO worker.py:1509 -- Started a local Ray instance. View the dashboard at http://127.0.0.1:8265 


(compute_qc_stats_ray pid=2763038) 2022-09-21 13:52:20,149 cisTopic     INFO     Reading VIB_hydrop_1.FIXEDCELLS
(compute_qc_stats_ray pid=2763038) 2022-09-21 13:53:16,617 cisTopic     INFO     Computing barcode rank plot for VIB_hydrop_1.FIXEDCELLS
(compute_qc_stats_ray pid=2763038) 2022-09-21 13:53:16,617 cisTopic     INFO     Counting fragments
(compute_qc_stats_ray pid=2763038) 2022-09-21 13:53:18,138 cisTopic     INFO     Marking barcodes with more than 10
(compute_qc_stats_ray pid=2763038) 2022-09-21 13:53:18,178 cisTopic     INFO     Returning plot data
(compute_qc_stats_ray pid=2763038) 2022-09-21 13:53:18,224 cisTopic     INFO     Returning valid barcodes
(compute_qc_stats_ray pid=2763038) 2022-09-21 13:53:19,221 cisTopic     INFO     Computing duplicate rate plot for VIB_hydrop_1.FIXEDCELLS
(compute_qc_stats_ray pid=2763038) 2022-09-21 13:53:21,763 cisTopic     INFO     Return plot data
(compute_qc_stats_ray pid=2763038) 2022-09-21 13:53:21,865 cisTopic     INFO     Computing

2022-09-21 13:57:26,348	INFO worker.py:1509 -- Started a local Ray instance. View the dashboard at http://127.0.0.1:8265 


(compute_qc_stats_ray pid=2763549) 2022-09-21 13:57:30,083 cisTopic     INFO     Reading VIB_hydrop_2.FIXEDCELLS
(compute_qc_stats_ray pid=2763549) 2022-09-21 13:58:43,568 cisTopic     INFO     Computing barcode rank plot for VIB_hydrop_2.FIXEDCELLS
(compute_qc_stats_ray pid=2763549) 2022-09-21 13:58:43,568 cisTopic     INFO     Counting fragments
(compute_qc_stats_ray pid=2763549) 2022-09-21 13:58:45,362 cisTopic     INFO     Marking barcodes with more than 10
(compute_qc_stats_ray pid=2763549) 2022-09-21 13:58:45,427 cisTopic     INFO     Returning plot data
(compute_qc_stats_ray pid=2763549) 2022-09-21 13:58:45,459 cisTopic     INFO     Returning valid barcodes
(compute_qc_stats_ray pid=2763549) 2022-09-21 13:58:46,657 cisTopic     INFO     Computing duplicate rate plot for VIB_hydrop_2.FIXEDCELLS
(compute_qc_stats_ray pid=2763549) 2022-09-21 13:58:49,587 cisTopic     INFO     Return plot data
(compute_qc_stats_ray pid=2763549) 2022-09-21 13:58:49,669 cisTopic     INFO     Computing